Pour créer les config files

(bgp pas encore fonctionnel)

In [ ]:
import json
import sys
import time
from concurrent.futures import ThreadPoolExecutor

base_config = [
    "version 15.2",
    "service timestamps debug datetime msec",
    "service timestamps log datetime msec",
    "boot-start-marker",
    "boot-end-marker",
    "no aaa new-model",
    "no ip icmp rate-limit unreachable",
    "ip cef",
    "no ip domain lookup",
    "ipv6 unicast-routing",
    "ipv6 cef",
    "multilink bundle-name authenticated",
    "ip tcp synwait-time 5",
    "ip forward-protocol nd",
    "no ip http server",
    "no ip http secure-server",
    "control-plane",
    "line con 0",
    " exec-timeout 0 0",
    " privilege level 15",
    " logging synchronous",
    " stopbits 1",
    "line aux 0",
    " exec-timeout 0 0",
    " privilege level 15",
    " logging synchronous",
    " stopbits 1",
    "line vty 0 4",
    " login",
    "end"]


def main(topology):
    with ThreadPoolExecutor() as executor:
        futures = []
        subnet_dict = give_subnet_number(topology)
        subnet_interconnexion_dict = give_subnet_interconnexion(topology, subnet_dict)
        for AS in topology:
            for router in topology[AS]['routers']:
                future = executor.submit(process_router, router, AS, subnet_dict[AS], subnet_interconnexion_dict[AS], topology)
                futures.append(future)

        for future in futures:
            future.result()


def process_router(router, AS, subnet_dict, subnet_interconnexion_dict, topology):
    create_base_cfg(router, base_config)
    create_loopback_interface(router, topology[AS])
    create_router_interfaces(router, AS, topology[AS], subnet_dict)
    activate_protocols(router, AS, topology[AS], subnet_interconnexion_dict,subnet_dict)


def insert_cfg_line(router, index_line, data):
    # insert une information 'data' à la ligne 'line'.
    with open(f'i{router[1:]}_startup-config.cfg', 'r') as file:
        lines = file.readlines()
        lines.insert(index_line, data)
    with open(f'i{router[1:]}_startup-config.cfg', 'w') as file:
        file.writelines(lines)


def create_base_cfg(router, base_config):
    # créer pour un routeur, son fichier cfg de base à partir d'une liste 'base config', et insert l'hostname.
    with open(f'i{router[1:]}_startup-config.cfg', 'w') as file:
        for entry in base_config:
            file.write(entry + '\n')
    insert_cfg_line(router, 3, f"hostname {router}\n")


def find_index(router, line):
    # Trouve l'indice correcte pour inserer une nouvelle ligne juste après la ligne 'line'.
    index_line = 1
    with open(f'i{router[1:]}_startup-config.cfg', 'r') as file:
        lines = file.readline()
        while lines != line:
            lines = file.readline()
            index_line += 1
    return index_line


def create_router_interfaces(router, AS, as_topology, subnet_dict):
    # insert dans le cfg du routeur toutes les ses interfaces et leurs ipv6 correspondantes.
    index_line = find_index(router, line="ip tcp synwait-time 5\n")

    for neighbor in as_topology['routers'][router].keys():
        if router[1:] < neighbor[1:]:
            subnet_index = subnet_dict[(router, neighbor)]
            router_index = 1
        else:
            subnet_index = subnet_dict[(neighbor, router)]
            router_index = 2

        insert_cfg_line(router, index_line,
                        f"interface {as_topology['routers'][router][neighbor]}\n no ip address\n negotiation auto\n ipv6 address {as_topology['address']}{subnet_index}::{router_index}{as_topology['subnet_mask']}\n ipv6 enable\n")
        index_line += 5


def give_subnet_number(as_topology):  # Crée le tableau des subnets pour tout le json
    subnet_dict = dict()
    for AS in as_topology:
        subnet_dict[AS] = dict()
        subnet_number = 1
        for router in as_topology[AS]['routers']:
            for neighbor in as_topology[AS]['routers'][router]:
                if router[1:] < neighbor[1:]:
                    subnet_dict[AS][(router, neighbor)] = subnet_number
                    subnet_number += 1
    return subnet_dict

def last_entries_subnet(subnet_dict):
    last_entry = dict()
    for AS in subnet_dict:
        last_entry[AS] = list(subnet_dict[AS].values())[-1]
    return last_entry
def give_subnet_interconnexion(as_topology, subnet_dict):  # Crée le tableau des subnets des interconnexion entre AS pour tout le json
    subnet_interconnexion_dict = dict()
    last_entries = last_entries_subnet(subnet_dict)
    for AS in as_topology:
        subnet_interconnexion_dict[AS] = dict()

    for AS in as_topology:
        for AS_neighbor in as_topology[AS]['neighbor']:
            for router1 in as_topology[AS]['neighbor'][AS_neighbor]:
                for router2 in as_topology[AS]['neighbor'][AS_neighbor][router1]:
                    if (router1, router2) not in subnet_interconnexion_dict[AS].keys() and (router2, router1) not in subnet_interconnexion_dict[AS].keys():
                        if AS < AS_neighbor:
                            subnet_interconnexion_dict[AS][(router1, router2)] = str(int(AS[3:])) + str((last_entries[AS] + 1)) + "::1"
                            subnet_interconnexion_dict[AS_neighbor][(router2, router1)] = str(int(AS[3:])) + str((last_entries[AS] + 1)) + "::2"
                        else:
                            subnet_interconnexion_dict[AS][(router1, router2)] = str(int(AS_neighbor[3:])) + str((last_entries[AS] + 1)) + "::1"
                            subnet_interconnexion_dict[AS_neighbor][(router2, router1)] = str(int(AS_neighbor[3:])) + str((last_entries[AS] + 1)) + "::2"
                        last_entries[AS] += 1
    return subnet_interconnexion_dict


def is_rip(as_topology):
    # retourne True si RIP est à activer, False sinon.
    return True if as_topology["protocol"] == "RIP" else False


def is_ospf(as_topology):
    # retourne True si RIP est à activer, False sinon
    return True if as_topology["protocol"] == "OSPF" else False


def activate_protocols(router, AS, as_topology, subnet_interconnexion_dict,subnet_dict):
    # active tous les protocols d'un routeur.
    router_id = give_router_id(router)
    if is_ospf(as_topology):
        activate_ospf(router, as_topology, router_id)
    elif is_rip(as_topology):
        activate_rip(router, as_topology)
    activate_bgp(router, AS, as_topology, subnet_interconnexion_dict,subnet_dict)


def activate_ospf(router, as_topology, router_id):
    # active OSPF sur le routeur.
    index_line = find_index(router, "no ip http secure-server\n")
    insert_cfg_line(router, index_line, f"ipv6 router ospf 1\n router-id {router_id}\n")
    for interface in as_topology["routers"][router].values():
        index_line = find_index(router, f"interface {interface}\n") + 4
        insert_cfg_line(router, index_line, " ipv6 ospf 1 area 0\n")

    if is_border_routers(router, as_topology):

        index_line = find_index(router, "ip forward-protocol nd\n") - 1
        insert_cfg_line(router, index_line, "router ospf 1\n")
        index_line = find_index(router, f" router-id {router_id}\n")
        for AS_neighbor in as_topology["neighbor"]:
            if router in as_topology["neighbor"][AS_neighbor].keys():
                for interface in as_topology["neighbor"][AS_neighbor][router].values():
                    insert_cfg_line(router, index_line, f" passive-interface {interface}\n")
                    

def activate_rip(router, as_topology):
    # active RIP sur le routeur.
    rip_process_name = "process"
    index_line = find_index(router, "no ip http secure-server\n")
    insert_cfg_line(router, index_line, f"ipv6 router rip {rip_process_name}\n redistribute connected\n")
    for interface in as_topology["routers"][router].values():
        index_line = find_index(router, f"interface {interface}\n") + 4
        insert_cfg_line(router, index_line, f" ipv6 rip {rip_process_name} enable\n")


def give_router_id(router):
    x = router[1:]
    return f"{x}.{x}.{x}.{x}"


def get_subnet_interconnexion(subnet_interconnexion_dict, routeur1, routeur2):
    return subnet_interconnexion_dict[(routeur1, routeur2)] or subnet_interconnexion_dict[(routeur2, routeur1)]


def activate_bgp(routeur, AS, as_topology, subnet_interconnexion_dict, subnet_dict):
    
    index_line = find_index(routeur, "ip forward-protocol nd\n") - 1
    if is_border_routers(routeur, as_topology):

        for AS_neighbor in as_topology["neighbor"]:
            for neighborRouter in as_topology["neighbor"][AS_neighbor]:
                if neighborRouter == routeur:
                    for neighborRouter2 in as_topology["neighbor"][AS_neighbor][neighborRouter]:
                        insert_cfg_line(routeur, index_line,
                                        f"interface {as_topology['neighbor'][AS_neighbor][neighborRouter][neighborRouter2]}\n no ip address\n negotiation auto\n ipv6 address {as_topology['address'][:-1]}{get_subnet_interconnexion(subnet_interconnexion_dict, routeur, neighborRouter2)}{as_topology['subnet_mask']}\n ipv6 enable\n")
                        index_line += 5
                        if is_ospf(as_topology):
                            insert_cfg_line(routeur, index_line," ipv6 ospf 1 area 0\n")
                            index_line +=1
                                            
    insert_cfg_line(routeur, index_line,
                    f"router bgp 10{AS[3:]}\n bgp router-id {give_router_id(routeur)}\n bgp log-neighbor-changes\n no bgp default ipv4-unicast\n")
    index_line += 4
    #neighbor autre as
    for AS_neighbor in as_topology["neighbor"]:
        if is_border_routers(routeur, as_topology):
           
            for subnet in subnet_interconnexion_dict:
                if subnet[0] == routeur:
                    if int(AS[3:]) < int(AS_neighbor[3:]):
                        network_tail = subnet_interconnexion_dict[subnet][:-1]+"2"
                        
                    else:
                        network_tail = subnet_interconnexion_dict[subnet][:-1]+"1"
                        

            insert_cfg_line(routeur, index_line, f" neighbor {as_topology['address'][:-1]}{network_tail} remote-as 10{AS_neighbor[3:]}\n")
            index_line +=1

    neighborConf = ""
    index_sum = 0
    for router in as_topology["routers"]:
        if router != routeur:
            neighborConf += f" neighbor 2001::{router[1:]} remote-as 10{AS[3:]}\n neighbor 2001::{router[1:]} update-source Loopback0\n"
            index_sum += 2
    insert_cfg_line(routeur, index_line, neighborConf)
    index_line += index_sum
    insert_cfg_line(routeur, index_line, f" address-family ipv4\n exit-address-family\n address-family ipv6\n")
    index_line += 3
    neighborConf = ""
    index_sum = 0

    for router in as_topology["routers"]:
        if router != routeur:
            neighborConf += f"  neighbor 2001::{router[1:]} activate\n"
            index_sum += 1
    if is_border_routers(routeur, as_topology):
        insert_cfg_line(routeur, index_line, f"  neighbor {as_topology['address'][:-1]}{network_tail} activate\n")
        index_line +=1

    insert_cfg_line(routeur, index_line, neighborConf)
    index_line += index_sum
    if is_border_routers(routeur, as_topology):
        
        create_networks(as_topology,routeur, subnet_dict, index_line)
        index_line +=1
    insert_cfg_line(routeur, index_line, " exit-address-family\n")


def is_border_routers(router, as_topology):
    state = False
    for AS_neighbor in as_topology["neighbor"]:
        if router in as_topology["neighbor"][AS_neighbor].keys():
            state = True
    return state

def create_networks(as_topology, router, subnet_dict, index_line):
    
    for AS_neighbor in as_topology["neighbor"]:        
        for AS in as_topology["pingable"]:
            if AS == AS_neighbor:
                network = as_topology["pingable"][AS]
                insert_cfg_line(router, index_line, f"  network {as_topology['address']}{subnet_dict[network[0][0], network[0][1]]}::{as_topology['subnet_mask']}\n" )
                
                    

def create_loopback_interface(router, as_topology):
    index_line = find_index(router, line="ip tcp synwait-time 5\n")
    insert_cfg_line(router, index_line, f"interface Loopback0\n no ip address\n ipv6 address 2001::{router[1:]}/128\n")
    index_line += 3
    if is_ospf(as_topology):
        insert_cfg_line(router, index_line, " ipv6 ospf 1 area 0\n")


if __name__ == "__main__":
    start = time.time()
    with open("new_intends.json", 'r') as file:
        topology = json.load(file)
    main(topology)
    end = time.time()
    print(end - start)

Pour move les fichiers crées

In [1]:
import os
import shutil
import time

def get_number_file(nom_fichier):
    if nom_fichier in correct_files:
        return int(nom_fichier[1:2])
    else:
        return None

def compare_and_paste(dossier_source, dossier_destination):
    noms_fichiers = os.listdir(os.path.join(os.getcwd(), dossier_destination))
    for nom_fichier in noms_fichiers:
        numero_fichier = get_number_file(nom_fichier)
        print(nom_fichier, numero_fichier)
        if numero_fichier is not None:
            chemin_fichier_source = os.path.join(dossier_source, nom_fichier)
            chemin_fichier_destination = os.path.join(dossier_destination, f"i{numero_fichier}_startupconfig.cfg")

            try:
                if os.path.exists(chemin_fichier_destination):
                    os.remove(chemin_fichier_destination)
                shutil.copy2(chemin_fichier_source, dossier_destination)
                print(f"Le fichier {nom_fichier} a été déplacé vers {dossier_destination}")
            except FileNotFoundError:
                print(f"Le fichier {nom_fichier} n'existe pas.")
            except PermissionError:
                print(f"Vous n'avez pas la permission de déplacer le fichier {nom_fichier}.")


start = time.time()

path_correct_files = os.path.join(os.getcwd())
correct_files = os.listdir(path_correct_files)

# Dossier GNS3
path_dossier_principal = r"C:\Users\User\GNS3\projects\quatorze_routeurs\project-files\dynamips"
dossiers = os.listdir(path_dossier_principal)

for nom_dossier in dossiers:
    path_dossier_destination = os.path.join(path_dossier_principal, nom_dossier, "configs")
    if os.path.exists(path_dossier_destination):
        compare_and_paste(path_correct_files, path_dossier_destination)

end = time.time()
print("temps d'execution :", end-start)

i13_startup-config.cfg 1
Le fichier i13_startup-config.cfg a été déplacé vers C:\Users\User\GNS3\projects\quatorze_routeurs\project-files\dynamips\24a179e3-a1cd-4b47-b0f3-eaca5aa9b4d6\configs
i7_startup-config.cfg 7
Le fichier i7_startup-config.cfg a été déplacé vers C:\Users\User\GNS3\projects\quatorze_routeurs\project-files\dynamips\3ffcbd07-6087-4efb-a4f4-4932529f3252\configs
i11_startup-config.cfg 1
Le fichier i11_startup-config.cfg a été déplacé vers C:\Users\User\GNS3\projects\quatorze_routeurs\project-files\dynamips\4754abd0-0297-4741-8538-6cffa7e9fdf9\configs
i9_startup-config.cfg 9
Le fichier i9_startup-config.cfg a été déplacé vers C:\Users\User\GNS3\projects\quatorze_routeurs\project-files\dynamips\53096d43-7045-46ed-b5f4-5597e4cb2ea6\configs
i2_startup-config.cfg 2
Le fichier i2_startup-config.cfg a été déplacé vers C:\Users\User\GNS3\projects\quatorze_routeurs\project-files\dynamips\7c59d511-dc5c-4836-a320-4fb9395ec9d6\configs
i10_startup-config.cfg 1
Le fichier i10_startu

Tests

In [9]:
import json
import time
from concurrent.futures import ThreadPoolExecutor

base_config = [
    "version 15.2",
    "service timestamps debug datetime msec",
    "service timestamps log datetime msec",
    "boot-start-marker",
    "boot-end-marker",
    "no aaa new-model",
    "no ip icmp rate-limit unreachable",
    "ip cef",
    "no ip domain lookup",
    "ipv6 unicast-routing",
    "ipv6 cef",
    "multilink bundle-name authenticated",
    "ip tcp synwait-time 5",
    "ip forward-protocol nd",
    "no ip http server",
    "no ip http secure-server",
    "control-plane",
    "line con 0",
    " exec-timeout 0 0",
    " privilege level 15",
    " logging synchronous",
    " stopbits 1",
    "line aux 0",
    " exec-timeout 0 0",
    " privilege level 15",
    " logging synchronous",
    " stopbits 1",
    "line vty 0 4",
    " login",
    "end"
]


def main(topology):
    with ThreadPoolExecutor() as executor:
        futures = []
        subnet_dict = give_subnet_number(topology)
        for AS in topology:
            for router in topology[AS]['routers']:
                future = executor.submit(process_router, router, AS, subnet_dict[AS], topology)
                futures.append(future)

        for future in futures:
            future.result()


def process_router(router, AS, subnet_dict, topology):
    create_base_cfg(router, base_config)
    create_loopback_interface(router, topology[AS])
    create_router_interfaces(router, AS, topology[AS], subnet_dict)


def insert_cfg_line(router, index_line, data):
    with open(f'i{router[1:]}_startup-config.cfg', 'r') as file:
        lines = file.readlines()
        lines.insert(index_line, data)
    with open(f'i{router[1:]}_startup-config.cfg', 'w') as file:
        file.writelines(lines)


def create_base_cfg(router, base_config):
    with open(f'i{router[1:]}_startup-config.cfg', 'w') as file:
        for entry in base_config:
            file.write(entry + '\n')
    insert_cfg_line(router, 3, f"hostname {router}\n")


def find_index(router, line):
    index_line = 1
    with open(f'i{router[1:]}_startup-config.cfg', 'r') as file:
        lines = file.readline()
        while lines != line:
            lines = file.readline()
            index_line += 1
    return index_line


def create_router_interfaces(router, AS, as_topology, subnet_dict):
    index_line = find_index(router, line="ip tcp synwait-time 5\n")

    for neighbor in as_topology['routers'][router].keys():
        if router[1:] < neighbor[1:]:
            subnet_index = subnet_dict[(router, neighbor)]
            router_index = 1
        else:
            subnet_index = subnet_dict[(neighbor, router)]
            router_index = 2

        insert_cfg_line(router, index_line,
                        f"interface {as_topology['routers'][router][neighbor]}\n no ip address\n negotiation auto\n ipv6 address {as_topology['address']}{subnet_index}::{router_index}{as_topology['subnet_mask']}\n ipv6 enable\n")
        index_line += 5


def give_subnet_number(as_topology):
    subnet_dict = dict()
    for AS in as_topology:
        subnet_dict[AS] = dict()
        subnet_number = 1
        for router in as_topology[AS]['routers']:
            for neighbor in as_topology[AS]['routers'][router]:
                if router[1:] < neighbor[1:]:
                    subnet_dict[AS][(router, neighbor)] = subnet_number
                    subnet_number += 1
    return subnet_dict


def create_loopback_interface(router, as_topology):
    index_line = find_index(router, line="ip tcp synwait-time 5\n")
    insert_cfg_line(router, index_line, f"interface Loopback0\n no ip address\n ipv6 address 2001::{router[1:]}/128\n")


if __name__ == "__main__":
    start = time.time()
    with open("new_intends.json", 'r') as file:
        topology = json.load(file)
    main(topology)
    end = time.time()
    print(end - start)


0.10155749320983887


In [10]:
import json

with open("new_intends.json", 'r') as file :
    topology = json.load(file)

def give_subnet_number(as_topology):
    subnet_dict = dict()
    for AS in as_topology:
        subnet_dict[AS] = dict()
        subnet_number = 1
        for router in as_topology[AS]['routers']:
            for neighbor in as_topology[AS]['routers'][router]:
                if router[1:] < neighbor[1:]:
                    subnet_dict[AS][(router, neighbor)] = subnet_number
                    subnet_number += 1
    return subnet_dict

a = give_subnet_number(topology)
print(a)

{'AS_1': {('R1', 'R2'): 1, ('R1', 'R3'): 2, ('R2', 'R3'): 3, ('R2', 'R4'): 4, ('R3', 'R5'): 5, ('R4', 'R5'): 6, ('R4', 'R6'): 7, ('R4', 'R7'): 8, ('R5', 'R6'): 9, ('R5', 'R7'): 10}, 'AS_2': {('R10', 'R8'): 1, ('R10', 'R9'): 2, ('R10', 'R11'): 3, ('R10', 'R12'): 4, ('R11', 'R8'): 5, ('R11', 'R9'): 6, ('R11', 'R13'): 7, ('R12', 'R13'): 8, ('R12', 'R14'): 9, ('R13', 'R14'): 10}}


### New version

In [1]:
import os

def insert_line(router: str, index_line: int, data: str) -> None:
    """
    For a given router, insert the data at indexline in its config file
    """
    filename = f"i{router[1::]}_startup_config.cfg"

    # Lire, insérer, et écrire les lignes
    with open(filename, 'r') as file:
        lines = file.readlines()
        lines.insert(index_line, data)

    with open(filename, 'w') as file:
        file.writelines(lines)


In [2]:
def find_index(router : str, line : str) -> int :
    """ 
    For a given router, finds the index of a given line in its config file
    """
    current_index = 1
    with open(f'i{router[1:]}_startup-config.cfg', 'r') as file:
        # Browses the lines to find the wanted one
        lines = file.readline()
        while lines != line:
            lines = file.readline()
            current_index += 1
    return current_index

In [3]:
def create_base_cfg(base_config : list, router : str) -> None :
    """
    Creates the base config file named iX_startup-config.cfg
    With X the number/name of the router

    The file contains all the lines form the base_config list plus the hostname of the router (= number/name)
    """

    filename = f"i{router[1::]}_startup_config.cfg"

    # Créer le fichier s'il n'existe pas
    if not os.path.exists(filename):
        with open(filename, 'w') as file:
            file.write('')  # Fichier vide
            
    # Writes the base_config in the config file
    with open(f'i{router[1:]}_startup-config.cfg', 'w') as file :
        for entry in base_config :
            file.write(entry + '\n')
    # Writes the hostname in the config file
    insert_line(router, 3, f"hostname {router}\n")   

In [ ]:
def create_loopback_interface(router : str) -> None :
    """ 
    Insert the loopback lines at the right place in the config file of a given router
    """
    # Finds the index of where to insert the loopback part
    index_line = find_index(router, "ip tcp synwait-time 5\n")
    #Insert the loopback part 
    insert_line(router, index_line, f"interface Loopback0\n no ip address\n ipv6 address 2001::{router[1:]}/128\n")

In [ ]:
def give_subnet_number(topology : dict) -> dict :
    """ 
    Creates a dict associating a unique number to every physical link in the network

    Exemaple : {'AS_1': {('R1', 'R2'): 1, ('R1', 'R3'): 2, ... }
    """
    subnet_dict = {}
    # Iterate over each AS
    for AS in topology :
        subnet_dict[AS] = {}
        subnet_number = 1
        # Iterate over each router of the current AS
        for router in topology[AS]['routers'] :
            # Iterate over each neighbor of the current router
            for neighbor in topology[AS]['routers'][router] :
                # To avoid duplicates, ensure the router with the smaller numeric suffix comes first
                if router[1:] < neighbor[1:] :
                    subnet_dict[AS][(router, neighbor)] = subnet_number
                    subnet_number += 1
    return subnet_dict

In [ ]:
def create_interfaces(router : str, topology : dict, ) -> None :
    """
    Generate the interfaces with the correct IPv6 addresses for each routrer of each AS 
    
    Exemaple : 
        interface GigabitEthernet1/0
        no ip address
        negotiation auto
        ipv6 address 2001:192:168:11::1/64
        ipv6 enable
    """
    #Creates the subnet_dict
    subnet_dict = give_subnet_number(topology)

    #Finds the line where to insert the interface
    index_line = find_index(router, line="ip tcp synwait-time 5\n")

    #Iterate over ...
    for neighbor in topology['routers'][router].keys() :
        # To ensure it's in the correct order
        if router[1:] < neighbor[1:] :
            subnet_index = subnet_dict[(router, neighbor)]
            router_index = 1
        else :
            subnet_index = subnet_dict[(neighbor, router)]
            router_index = 2
        
        #Insert the lines in the config files
        insert_cfg_line(router, index_line,
            f"interface {topology['routers'][router][neighbor]}\n"  # Interface name
            f" no ip address\n"  # Disable IPv4 addressing
            f" negotiation auto\n"  # Enable automatic negotiation for the interface
            f" ipv6 address {topology['address']}{subnet_index}::{router_index}{topology['subnet_mask']}\n"  # Assign an IPv6 address
            f" ipv6 enable\n"  # Enable IPv6 on the interface
        )

        # Increment the index line to ensure the next configuration is inserted at the correct position
        index_line += 5
    

In [2]:
import json

with open("new_intends.json", 'r') as file :
    topology = json.load(file)

for AS in topology :
    for router in topology[AS]['routers'] :
        print(router[1:])
        # create_base_cfg(["line 1", "line 2", "line 3", "line 4", "line 5"], router)

1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [5]:
import os

path_dossier_principal = "C:/Users/User/GNS3/projects/quatorze_routeurs/project-files/dynamips"

if not os.path.exists(path_dossier_principal):
    print(f"Le chemin {path_dossier_principal} n'existe pas. Vérifie le chemin.")
else:
    dossiers = os.listdir(path_dossier_principal)
    print(dossiers)

['24a179e3-a1cd-4b47-b0f3-eaca5aa9b4d6', '3ffcbd07-6087-4efb-a4f4-4932529f3252', '4754abd0-0297-4741-8538-6cffa7e9fdf9', '53096d43-7045-46ed-b5f4-5597e4cb2ea6', '7c59d511-dc5c-4836-a320-4fb9395ec9d6', '8b0ddcd1-6e7a-4fb2-85ad-8de0024e0397', '930cd219-e161-459b-8ea9-e4d3a44241c7', '9bcb1ecb-f964-4c18-b0a7-539133773c7e', 'a786893f-017c-4f7d-a439-1b396ec48bb8', 'b0a976c4-c15f-4fd1-bca6-93778602b086', 'b733807e-0d8e-4908-8707-4099a98d619e', 'c7200-advipservicesk9-mz.152-4.S5.image-512.ghost', 'dynamips_i10_log.txt', 'dynamips_i10_stdout.txt', 'dynamips_i11_log.txt', 'dynamips_i11_stdout.txt', 'dynamips_i12_log.txt', 'dynamips_i12_stdout.txt', 'dynamips_i13_log.txt', 'dynamips_i13_stdout.txt', 'dynamips_i14_log.txt', 'dynamips_i14_stdout.txt', 'dynamips_i1_log.txt', 'dynamips_i1_stdout.txt', 'dynamips_i2_log.txt', 'dynamips_i2_stdout.txt', 'dynamips_i3_log.txt', 'dynamips_i3_stdout.txt', 'dynamips_i4_log.txt', 'dynamips_i4_stdout.txt', 'dynamips_i5_log.txt', 'dynamips_i5_stdout.txt', 'dynam